In [ ]:
# @title Importing my gemini API key
api_key_file = '/content/key.txt'
with open(api_key_file, 'r') as file:
    api_key = file.read().strip()

In [ ]:
# @title Necessary Imports
import requests
from google import genai
from pydantic import BaseModel
from PIL import Image
import json

In [ ]:
# @title Defining the food content parameters that we'll need for the next step
food_content = {"Calories","Saturated Fat","Trans Fat","Naturally occurring Sugar","Refined sugar","Salt"}

In [ ]:
# @title JSON structure in which the model should return the output.
class IngredientQuantity(BaseModel):
    ingredient: str
    quantity: float
    unit: str
    logic_used_to_calculate_quantity: str

In [ ]:
# @title Calling the api model with inputs: food image.
def what_is_this_food(food_image):

    client = genai.Client(api_key=api_key)

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[food_image, "What is this food item called? Just give me the food item name and nothing else."],
    )

    return response.text

In [ ]:
# @title Calling the api model with inputs: food item name. Uses ingridients whose quantity is needed and output json format internally.
def get_food_analysis(food_description):

    prompt = f"Analyze the food item: '{food_description}' and give me the amount of every ingredient in list: {food_content} to create 100 grams of '{food_description}'."

    client = genai.Client(api_key=api_key)

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            "response_schema": list[IngredientQuantity]
        }
    )

    # Storing the content generated in a JSON format, which we can use later on in the code, to get the content of every ingredient.
    food_data = json.loads(response.text)
    return food_data

In [ ]:
# @title Sample usage: Prints the food's name.
image = Image.open("/content/food_image.jpg")
food_description = what_is_this_food(image)

In [ ]:
# @title Sample usage: Prints the food item name.
print(food_description)
food_analysis_data = get_food_analysis(food_description)

Chole Bhature



In [ ]:
# @title Sample usage: Prints the content of ingridients in 100gm quantity of the stated food item.
print(food_analysis_data)

[{'ingredient': 'Naturally occurring Sugar', 'quantity': 1.5, 'unit': 'g', 'logic_used_to_calculate_quantity': 'Estimated based on the carbohydrate content of chickpeas, flour, and other vegetables used in the dish. Assumed a percentage of these carbohydrates are naturally occurring sugars.'}, {'ingredient': 'Salt', 'quantity': 1.2, 'unit': 'g', 'logic_used_to_calculate_quantity': 'Estimated based on common recipes for Chole Bhature, accounting for salt added during the preparation of both the chole (chickpea curry) and bhature (fried bread).'}, {'ingredient': 'Trans Fat', 'quantity': 0.3, 'unit': 'g', 'logic_used_to_calculate_quantity': 'Trans fat can form during the deep-frying process of the bhature. The quantity is estimated based on typical trans fat levels in fried foods.'}, {'ingredient': 'Saturated Fat', 'quantity': 6.0, 'unit': 'g', 'logic_used_to_calculate_quantity': 'Estimated based on the use of oil (often ghee or vegetable oil) in the preparation of both the chole and bhat

In [ ]:
for index in range(len(food_analysis_data)):
    print(f"{food_analysis_data[index]['ingredient']}: {food_analysis_data[index]['quantity']} {food_analysis_data[index]['unit']}\n")

Naturally occurring Sugar: 1.5 g

Salt: 1.2 g

Trans Fat: 0.3 g

Saturated Fat: 6.0 g

Calories: 350.0 kcal

Refined sugar: 0.5 g



In [ ]:
# @title Define Risk Thresholds
# food_content = {"Calories","Saturated Fat","Trans Fat","Naturally occurring Sugar","Refined sugar","Salt"}
risk_thresholds = {
    "Calories": [
        (150, 0.00),   # Low risk
        (300, 0.05),   # Moderate risk
        (450, 0.15),   # High risk
        (float('inf'), 0.25)  # Very high risk
    ],
    "Saturated Fat": [
        (3, 0.00),     # Low
        (6, 0.10),     # Moderate
        (9, 0.25),     # High
        (float('inf'), 0.40)  # Very High
    ],
    "Trans Fat": [
        (0.1, 0.00),   # Low
        (0.5, 0.20),   # Moderate
        (1.0, 0.35),   # High
        (float('inf'), 0.60)  # Very High
    ],
    "Refined sugar": [
        (5, 0.00),     # Low
        (12, 0.10),    # Moderate
        (20, 0.30),    # High
        (float('inf'), 0.50)  # Very High
    ],
    "Naturally occurring Sugar": [
        (5, 0.00),     # Low
        (15, 0.05),    # Moderate
        (25, 0.10),    # High
        (float('inf'), 0.15)  # Very High
    ],
    "Salt": [
        (0.3, 0.00),   # Low
        (0.6, 0.10),   # Moderate
        (1.2, 0.25),   # High
        (float('inf'), 0.40)  # Very High
    ]
}

In [ ]:
# @title Function to Get Component Risk
def get_component_risk(component_name, quantity, thresholds):
    component_thresholds = thresholds.get(component_name)
    if not component_thresholds:
        return 0.0  # Return 0 risk if component not in thresholds

    risk = 0.0
    # Assuming thresholds are sorted in ascending order
    for threshold, risk_percentage in component_thresholds:
        if quantity >= threshold:
            risk = risk_percentage
        else:
            break # Stop if quantity is not above the current threshold

    return risk

In [ ]:
# @title Calculate Combined Heart Risk
combined_risk_probability = 1.0
for item in food_analysis_data:
    ingredient = item['ingredient']
    quantity = item['quantity']

    # Only consider the components we have defined thresholds for
    if ingredient in risk_thresholds:
        component_risk = get_component_risk(ingredient, quantity, risk_thresholds)
        print(f"Quantity of {ingredient}: {quantity}\nRisk of {ingredient}: {component_risk}\n\n")
        combined_risk_probability *= (1 - component_risk)

total_heart_risk = 1 - combined_risk_probability

print(f"\nTotal estimated heart risk from consuming 100gms/100ml {food_description} is {total_heart_risk:.2%}")

Quantity of Naturally occurring Sugar: 1.5
Risk of Naturally occurring Sugar: 0.0


Quantity of Salt: 1.2
Risk of Salt: 0.25


Quantity of Trans Fat: 0.3
Risk of Trans Fat: 0.0


Quantity of Saturated Fat: 6.0
Risk of Saturated Fat: 0.1


Quantity of Calories: 350.0
Risk of Calories: 0.05


Quantity of Refined sugar: 0.5
Risk of Refined sugar: 0.0



Total estimated heart risk from consuming 100gms/100ml Chole Bhature
 is 35.88%


In [ ]:
import gradio as gr

# Main function
def analyze_food_image(image_path):
    if image_path is None:
        return "Please upload an image.", "", ""

    try:
        image = Image.open(image_path)
        food_description = what_is_this_food(image)
        food_analysis_data = get_food_analysis(food_description)

        combined_risk_probability = 1.0
        analysis_text = ""
        for item in food_analysis_data:
            ingredient = item['ingredient']
            quantity = item['quantity']
            unit = item['unit']
            analysis_text += f"{ingredient}: {quantity} {unit}\n"

            if ingredient in risk_thresholds:
                component_risk = get_component_risk(ingredient, quantity, risk_thresholds)
                combined_risk_probability *= (1 - component_risk)

        total_heart_risk = 1 - combined_risk_probability
        risk_text = f"Estimated Heart Risk from consuming 100g/ml of {food_description}: {total_heart_risk:.2%}"

        return food_description, analysis_text, risk_text

    except Exception as e:
        return f"An error occurred: {e}", "", ""

# Gradio app with custom CSS and light theme layout
with gr.Blocks() as demo:
    gr.HTML("""
        <style>
            body {
                background: #f3f4f6;
                font-family: 'Segoe UI', sans-serif;
                color: #333;
                padding: 20px;
            }

            h1 {
                font-size: 2.5em;
                color: #4CAF50;
                margin-bottom: 20px;
            }

            p {
                font-size: 1.2em;
                color: #555;
                margin-bottom: 30px;
            }

            .gr-box {
                border-radius: 10px;
                background-color: #ffffff;
                padding: 20px;
                box-shadow: 0px 4px 15px rgba(0,0,0,0.1);
                margin-bottom: 15px;
            }

            button {
                background-color: #4CAF50;
                color: white;
                font-size: 1.1em;
                padding: 12px 24px;
                border-radius: 8px;
                margin-top: 10px;
                width: 100%;
            }

            button:hover {
                background-color: #45a049;
            }

            .gr-row, .gr-column {
                margin-top: 20px;
            }

            .gr-image {
                border-radius: 10px;
                padding: 10px;
                background: #ffffff;
                box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
            }
        </style>

        <div style="text-align:center;">
            <h1>🥗 Food Health Analyzer</h1>
            <p>Upload your food image and instantly get a health analysis!</p>
        </div>
    """)

    with gr.Row():
        image_input = gr.Image(type="filepath", label="📷 Upload Food Image", elem_classes=["gr-box"])
        with gr.Column():
            food_description_output = gr.Textbox(label="🍔 Identified Food Item", lines=1, elem_classes=["gr-box"])
            food_analysis_output = gr.Textbox(label="🔍 Food Analysis (per 100g/ml)", lines=4, elem_classes=["gr-box"])
            risk_output = gr.Textbox(label="❤️ Estimated Heart Risk", lines=2, elem_classes=["gr-box"])

    analyze_button = gr.Button("🚀 Analyze Image")
    analyze_button.click(
        analyze_food_image,
        inputs=image_input,
        outputs=[food_description_output, food_analysis_output, risk_output]
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da9814a9303b963510.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
